<img src="https://admissions.ntust.edu.tw/front_index/images/logo.png" alt="drawing" width="600"/>
<hr>
<h1><center>MACHINE LEARNING : HOMEWORK 2</center></h1>
<h1><center>Data 1 : Wealth prediction</center></h1>
<h4>Hector LANDES - M10601810</h4>
<h4>Machine Learning - CS5087701</h4>

In [4]:
import pandas as pd
import numpy as np

In [5]:
path = r'C:\Users\Hector Landes\Desktop\Data Science Projects\20191005 Homework_1_ML\2. Prepared Data\adult.csv'

In [6]:
data = pd.read_csv (path, names=["age", "workclass", "fnlwgt", "education", "education_num", "marital_status",
                           "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", 
                                 "hours_per_week", "native_country", "wealth"],
                   index_col=False)

<ul>
<h3>Preprocessing from the HM1</3>
</ul>

In [7]:
data = data[(data.astype(str) != ' ?').all(axis=1)]

In [8]:
data = pd.get_dummies(data, columns=['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex'])

In [9]:
data['native_country'] = data['native_country'].str.strip()

In [10]:
data['native_country'] = data['native_country'].replace(['Japan', 'Cambodia', "Philippines", 
                                                         "India", "China", "Vietnam", "Taiwan", "Iran", 
                                                         "Hong", "Laos", "Thailand"], 'Asia')
data['native_country'] = data['native_country'].replace(["Germany", "England", "Italy", "Poland", "Portugal",
                                                         "Greece", "France", "Ireland", "Yugoslavia", "Hungary", 
                                                         "Scotland", "Holand-Netherlands"], 'Europe')
data['native_country'] = data['native_country'].replace(["Mexico","El-Salvador", "Cuba", "Jamaica", 'South', 
                                                         "Dominican-Republic", "Guatemala", "Columbia", "Haiti", 
                                                         "Nicaragua", "Peru", "Ecuador", "Trinadad&Tobago", "Honduras"], 
                                                        'South&Central America')
data['native_country'] = data['native_country'].replace(["United-States", 'Puerto-Rico', 'Canada',
                                                         'Outlying-US(Guam-USVI-etc)'], 'United_States')

In [11]:
data.rename(columns={'native_country':'native_region'}, inplace=True)

In [12]:
data = pd.get_dummies(data, columns=['native_region'])

In [13]:
non_zero_value = data.loc[data["capital_loss"] != 0]

In [14]:
data.loc[data["capital_loss"] != 0, ["capital_loss"]] = 1

In [15]:
non_zero_value = data.loc[data["capital_gain"] != 0]

In [16]:
data.loc[data["capital_gain"] != 0, ["capital_gain"]] = 1

In [17]:
data = data.drop(['fnlwgt'], axis=1)

In [18]:
data['bi_wealth'] = data.apply(lambda row: 1 if '>50K'in row['wealth'] else 0, axis=1)

In [19]:
data = data.drop(['wealth'], axis=1)

In [20]:
data.head()

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,...,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male,native_region_Asia,native_region_Europe,native_region_South&Central America,native_region_United_States,bi_wealth
0,39,13,1,0,40,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
1,50,13,0,0,13,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
2,38,9,0,0,40,0,0,1,0,0,...,0,0,1,0,1,0,0,0,1,0
3,53,7,0,0,40,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0
4,28,13,0,0,40,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,0


<ul>
<h3> X as the Feature Matrix </h3> 
</ul>

In [21]:
X = data[data.columns[:66]]
X[0:2]

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,...,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male,native_region_Asia,native_region_Europe,native_region_South&Central America,native_region_United_States
0,39,13,1,0,40,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
1,50,13,0,0,13,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,1


<ul>
<h3> Y as the Target Value </h3> 
</ul>

In [22]:
Y = data["bi_wealth"]
Y[0:5]

0    0
1    0
2    0
3    0
4    0
Name: bi_wealth, dtype: int64

In [27]:
#pip install keras
#pip install tensorflow
#pip install ann_visualizer

In [2]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from ann_visualizer.visualize import ann_viz;
import graphviz
import os
os.environ["PATH"] += os.pathsep + r'C:\Users\Hector Landes\graphviz-2.38\release\bin'

Using TensorFlow backend.


Define Keras Model

In [29]:
model = Sequential()
model.add(Dense(12, input_dim=66, activation='relu')) #Model expects rows of data with 66 variables #The first hidden layer has 12 nodes and uses the relu activation function
model.add(Dense(8, activation='relu')) #The seond hidden layer has 8 nodes and use relu activation function
model.add(Dense(1, activation='sigmoid')) #The output layer has one node and use the sigmoid activation function

Compile Keras Model

In [30]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Fit Keras Model

In [31]:
model.fit(X, Y, epochs=150, batch_size=10)

Epoch 1/150
30162/30162 [==============================] - 2s 70us/step - loss: 0.3913 - accuracy: 0.8205
Epoch 2/150
30162/30162 [==============================] - 2s 66us/step - loss: 0.3445 - accuracy: 0.8399
Epoch 3/150
30162/30162 [==============================] - 2s 65us/step - loss: 0.3422 - accuracy: 0.8403
Epoch 4/150
30162/30162 [==============================] - 2s 65us/step - loss: 0.3406 - accuracy: 0.8400
Epoch 5/150
30162/30162 [==============================] - 2s 64us/step - loss: 0.3393 - accuracy: 0.8416
Epoch 6/150
30162/30162 [==============================] - 2s 64us/step - loss: 0.3386 - accuracy: 0.8412
Epoch 7/150
30162/30162 [==============================] - 2s 64us/step - loss: 0.3364 - accuracy: 0.8414
Epoch 8/150
30162/30162 [==============================] - 2s 65us/step - loss: 0.3358 - accuracy: 0.8428
Epoch 9/150
30162/30162 [==============================] - 2s 68us/step - loss: 0.3357 - accuracy: 0.8426
Epoch 10/150
30162/30162 [====================

30162/30162 [==============================] - 2s 65us/step - loss: 0.3231 - accuracy: 0.8473
Epoch 78/150
30162/30162 [==============================] - 2s 65us/step - loss: 0.3236 - accuracy: 0.8465
Epoch 79/150
30162/30162 [==============================] - 2s 65us/step - loss: 0.3229 - accuracy: 0.8488
Epoch 80/150
30162/30162 [==============================] - 2s 66us/step - loss: 0.3231 - accuracy: 0.8485
Epoch 81/150
30162/30162 [==============================] - 2s 67us/step - loss: 0.3227 - accuracy: 0.8479
Epoch 82/150
30162/30162 [==============================] - 2s 66us/step - loss: 0.3231 - accuracy: 0.8486
Epoch 83/150
30162/30162 [==============================] - 2s 66us/step - loss: 0.3235 - accuracy: 0.8487
Epoch 84/150
30162/30162 [==============================] - 2s 66us/step - loss: 0.3230 - accuracy: 0.8482
Epoch 85/150
30162/30162 [==============================] - 2s 65us/step - loss: 0.3230 - accuracy: 0.8492
Epoch 86/150
30162/30162 [========================

Evaluate Keras Model

In [37]:
accuracy = model.evaluate(X, Y)
print('Accuracy: %.2f' % (accuracy[1]*100),'%')
print('Loss: %.2f' % (accuracy[0]*100),'%')

30162/30162 [==============================] - 0s 9us/step
Accuracy: 85.16 %
Loss: 31.84 %


Predictions

In [40]:
predictions = model.predict(X)
rounded = [round(x[0]) for x in predictions]

In [41]:
predictions = model.predict_classes(X)

In [58]:
for i in range(10):
    print('%d (expected %d)' % (predictions[i], Y[i]))

0 (expected 0)
1 (expected 0)
0 (expected 0)
0 (expected 0)
0 (expected 0)
1 (expected 0)
0 (expected 0)
0 (expected 1)
1 (expected 1)
1 (expected 1)


Plot the ANN

In [69]:
ann_viz(model, title="Wealth Prediction")

Maybe improve with this
<ul>
https://github.com/GoogleCloudPlatform/cloudml-samples/blob/master/census/keras/trainer/model.py
</ul>
<ul>
https://github.com/guillaume-chevalier/Predict-if-salary-is-over-50k-with-Keras/blob/master/.ipynb_checkpoints/Predict%20if%20salary%20is%20%3E50k-checkpoint.ipynb
</ul>
<ul>
https://towardsdatascience.com/get-started-with-deep-learning-using-keras-a45ee421f3ef
</ul>